## Cropped Models

In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

In [ ]:
!pip install timm
!pip install transformers timm ftfy

In [ ]:
# u have to get ride of the empty files or the model wont run
import os

folders = [
    "/content/DL_Temp/train/ai",
    "/content/DL_Temp/train/real",
    "/content/DL_Temp/test/ai",
    "/content/DL_Temp/test/real",
    "/content/DL_Temp/validation/ai",
    "/content/DL_Temp/validation/real"
]

count_removed = 0
for folder in folders:
  for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path) and f.lower().endswith(('.jpg', '.jpeg')):
        if os.path.getsize(path) == 0:
            os.remove(path)
            count_removed += 1
            # print(f"Deleted empty file: {path}")
print('Count of deleted files : ', count_removed)


In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

In [ ]:
size_check = {}
# verify that the images are the same size or not
# print(os.listdir('1_fake')[0:10])
for img in os.listdir('/content/DL_Temp/train/ai'):
    if '.ipynb_checkpoints' == img:
        continue
    images=Image.open('/content/DL_Temp/train/ai/'+img)
    # print(images.size)
    if images.size in size_check:
        size_check[images.size] += 1
    else:
        size_check[images.size] = 1
# sizes are not the same
print(list(size_check.items())[0:10])


In [ ]:
batch_sz = 32
learning_rate = 1e-4
epochs = 10
classes = 2
save_weights = "convnext_tiny_best.pth"
l_plot = "loss_curve.png"
acc_plot = "accuracy_curve.png"

In [ ]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()])

tf_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

tf_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()])

train_dset = datasets.ImageFolder('/content/DL_Temp/train', transform=tf_train)
val_dset   = datasets.ImageFolder('/content/DL_Temp/validation', transform=tf_eval)

print(f"Classes: {train_dset.classes}")
print(f"Total images: {len(train_dset)}")

train_loader = DataLoader(train_dset, batch_size=batch_sz, shuffle=True)
val_loader = DataLoader(val_dset, batch_size=batch_sz, shuffle=False)

In [ ]:
model = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
convnext_tiny = model.to(device)

print("Model ready on device:", device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnext_tiny.parameters(), lr=learning_rate)

In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='binary')
    recall    = recall_score(all_labels, all_preds, average='binary')
    f1        = f1_score(all_labels, all_preds, average='binary')

    return (running_loss / len(loader),correct / total,precision,recall,f1)


In [ ]:
metrics = {
    "train_loss": [], "val_loss": [],
    "train_acc": [],  "val_acc": [],
    "precision": [],  "recall": [],
    "f1": []
}

best_val_acc = 0

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(convnext_tiny, train_loader, optimizer, criterion)
    val_loss, val_acc, precision, recall, f1 = validate(convnext_tiny, val_loader, criterion)

    metrics["train_loss"].append(train_loss)
    metrics["val_loss"].append(val_loss)
    metrics["train_acc"].append(train_acc)
    metrics["val_acc"].append(val_acc)
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1"].append(f1)


    print(f"Epoch {epoch}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print(f"  Precision:  {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    print("-" * 60)


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(convnext_tiny.state_dict(), save_weights)
        print(f"Best model saved to: {save_weights}")

epochs_ = range(1, epochs + 1)
plt.figure()
plt.plot(epochs_, metrics["train_loss"], label="Train Loss")
plt.plot(epochs_, metrics["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.savefig(f'convnext_{l_plot}')
print(f"Saved loss plot: {l_plot}")
plt.close()


plt.figure()
plt.plot(epochs_, metrics["train_acc"], label="Train Accuracy")
plt.plot(epochs_, metrics["val_acc"], label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()
plt.savefig(f'convnext_{acc_plot}')
print(f"Saved accuracy plot: {acc_plot}")
plt.close()

## Model Vision Transformer

In [ ]:
torch.cuda.empty_cache()
save_weights = "vit_model_best.pth"
model = timm.create_model("vit_tiny_patch16_224", pretrained=True, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vit_model = model.to(device)

print("Model ready on device:", device)
metrics = {
    "train_loss": [], "val_loss": [],
    "train_acc": [],  "val_acc": [],
    "precision": [],  "recall": [],
    "f1": []
}

best_val_acc = 0

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(vit_model, train_loader, optimizer, criterion)
    val_loss, val_acc, precision, recall, f1 = validate(vit_model, val_loader, criterion)

    metrics["train_loss"].append(train_loss)
    metrics["val_loss"].append(val_loss)
    metrics["train_acc"].append(train_acc)
    metrics["val_acc"].append(val_acc)
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1"].append(f1)


    print(f"Epoch {epoch}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print(f"  Precision:  {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    print("-" * 60)


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(vit_model.state_dict(), save_weights)
        print(f"Best model saved to: {save_weights}")

epochs_ = range(1, epochs + 1)
plt.figure()
plt.plot(epochs_, metrics["train_loss"], label="Train Loss")
plt.plot(epochs_, metrics["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.savefig(l_plot)
print(f"Saved loss plot: {l_plot}")
plt.close()


plt.figure()
plt.plot(epochs_, metrics["train_acc"], label="Train Accuracy")
plt.plot(epochs_, metrics["val_acc"], label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()
plt.savefig(acc_plot)
print(f"Saved accuracy plot: {acc_plot}")
plt.close()


In [ ]:
torch.cuda.empty_cache()
save_weights = "Shufflenet_model_best.pth"
model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(1024, 2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vit_model = model.to(device)

print("Model ready on device:", device)
metrics = {
    "train_loss": [], "val_loss": [],
    "train_acc": [],  "val_acc": [],
    "precision": [],  "recall": [],
    "f1": []
}

best_val_acc = 0

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(vit_model, train_loader, optimizer, criterion)
    val_loss, val_acc, precision, recall, f1 = validate(vit_model, val_loader, criterion)

    metrics["train_loss"].append(train_loss)
    metrics["val_loss"].append(val_loss)
    metrics["train_acc"].append(train_acc)
    metrics["val_acc"].append(val_acc)
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1"].append(f1)


    print(f"Epoch {epoch}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print(f"  Precision:  {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    print("-" * 60)


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(vit_model.state_dict(), save_weights)
        print(f"Best model saved to: {save_weights}")

epochs_ = range(1, epochs + 1)
plt.figure()
plt.plot(epochs_, metrics["train_loss"], label="Train Loss")
plt.plot(epochs_, metrics["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.savefig(f'Shufflenet_{l_plot}')
print(f"Saved loss plot: {l_plot}")
plt.close()


plt.figure()
plt.plot(epochs_, metrics["train_acc"], label="Train Accuracy")
plt.plot(epochs_, metrics["val_acc"], label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()
plt.savefig(f'Shufflenet_{acc_plot}')
print(f"Saved accuracy plot: {acc_plot}")
plt.close()
